In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd

In [2]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES']

In [ ]:
from bidsify_helpers import make_bids_dirs, add_misc_bids_files

In [ ]:
make_bids_dirs()

In [ ]:
add_misc_bids_files()

In [3]:
timing_mats = glob.glob(raw_data_path + '/*/*/*.mat')

In [6]:
timing_mats[1]

'/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri/sub-609/ses-02/taskYN_subj609_day7_session4.mat'

In [114]:
tmp = sio.loadmat(timing_mats[1], squeeze_me=True)

In [115]:
tmp.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mydata', 'timing'])

In [116]:
timing = tmp['timing']

In [117]:
timing.dtype

dtype([('Begin', 'O'), ('onset', 'O'), ('feedbackOn', 'O'), ('crossStart', 'O'), ('crossEnd', 'O'), ('End', 'O'), ('session', 'O')])

In [120]:
# timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End']
timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End', 'session']
timing_vals = timing.item()
# pd.melt(pd.DataFrame(dict(zip(timing_keys, timing_vals)))).drop_duplicates().sort_values(by=['value'])
dict(zip(timing_keys, timing_vals))

{'Begin': array([4829.2565264, 5918.289145 ]),
 'onset': array([4839.2652949, 4844.0897413, 4847.5641997, 4850.3338168,
        4853.287047 , 4857.523724 , 4860.7381869, 4866.6126551,
        4872.0472269, 4875.3735859, 4878.6280222, 4881.4291583,
        4884.9991719, 4890.3753556, 4896.6903727, 4901.0748635,
        4905.9293101, 4912.7581683, 4916.2614128, 4920.9458604,
        4927.5385502, 4935.1956044, 4939.9000502, 4944.7745317,
        4948.9089877, 4951.9277449, 4955.1307451, 4958.2836415,
        4964.9065077, 4971.12095  , 4973.9981848, 4977.9226346,
        4980.847069 , 4987.7015054, 4992.2984769, 4995.9685252,
        4998.8329462, 5005.0974106, 5009.5693029, 5014.3522341,
        5017.9221555, 5023.0769553, 5026.471391 , 5029.3258374,
        5034.6102938, 5038.0547666, 5041.9492075, 5048.1936728,
        5052.6681381, 5058.8938494, 5064.0483054, 5069.9027427,
        5072.7395878, 5077.3939492, 5083.6483941, 5086.5428395,
        5089.3072623, 5092.4911885, 5095.8776659

In [88]:
timing.item()[1]

array([6986.0023222, 6989.3802566, 6992.8747549, 6995.7195133,
       7001.1082349, 7010.6164817, 7016.0548587, 7019.8250063,
       7022.8111097, 7029.4839832, 7038.7925626, 7052.3551285,
       7057.9269604, 7061.2214239, 7065.875885 , 7072.2222011,
       7078.0455596, 7084.3013526, 7089.4227619, 7093.4765063,
       7097.0298335, 7100.3495481, 7104.8440194, 7109.9917891,
       7117.4562517, 7120.9352597, 7125.4894696, 7130.1104402,
       7138.8518391, 7145.906304 , 7158.0028864, 7161.8873546,
       7168.2790769, 7172.1660054, 7177.9880621, 7185.811964 ,
       7189.0816682, 7193.0854082, 7195.8546159, 7199.7890735,
       7203.2835866, 7207.1817415, 7213.5710431, 7219.1261343,
       7222.4792749, 7225.2317847, 7231.9046245, 7234.85743  ,
       7238.8519067, 7242.2563792, 7245.8842409, 7260.7146679,
       7265.8527148, 7270.5570765, 7275.2947864, 7278.4143621,
       7281.6506679, 7285.8879265, 7290.8592123, 7293.8286275,
       7298.4662708, 7302.8407292, 7308.8251864, 7314.2

In [35]:
timing[0][0]['Begin']

array([[6975.9910616]])

In [37]:
timing['Begin'][0][0]

array([[6975.9910616]])

In [43]:
timing['onset'][0][0][0]

array([6986.0023222])

In [44]:
timing[0][0][0]['onset']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
tmp['timing'][0][0][0] #Begin

In [ ]:
tmp['timing'][0][0][5] #End

In [ ]:
tmp['timing'][0][0][1] #onset